In [2]:
import evdev

In [3]:
devices = [evdev.InputDevice(path) for path in evdev.list_devices()]

In [4]:
devices

[InputDevice('/dev/input/event14'), InputDevice('/dev/input/event7')]

In [5]:
for device in devices:
    print(device.path, device.name, device.phys)

/dev/input/event14 Logitech G29 Driving Force Racing Wheel usb-0000:00:14.0-2/input0
/dev/input/event7 LiteOn Lenovo Calliope USB Keyboard Consumer Control usb-0000:00:14.0-3/input1


In [44]:
import evdev
from evdev import ecodes, InputDevice

device = evdev.list_devices()[0]
evtdev = InputDevice(device)
val = 65535  # val \in [0,65535]
evtdev.write(ecodes.EV_FF, ecodes.FF_SPRING, val)

In [7]:
import time
import evdev
from evdev import ecodes, ff, InputDevice

device = evdev.list_devices()[0]
evtdev = InputDevice(device)